In [1]:
%load_ext autoreload
%autoreload 2

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
import sys
import os
sys.path.append('../')
import gp
import cPickle as pickle

/home/d/nolearn/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using gpu device 0: GeForce GTX TITAN (CNMeM is disabled, CuDNN 4007)
/home/d/nolearn/local/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [61]:
BASEDIR = '/home/d/netstatsPAPER/IPMLB/'
OUTDIR = '/home/d/GPSTUDY/'
merge_vis = BASEDIR+'dojo_merge_simuser_vis.p'
split_vis = BASEDIR+'dojo_split_simuser_vis.p'
merge_fixes = BASEDIR+'dojo_merge_simuser_fixes.p'
split_fixes = BASEDIR+'dojo_split_simuser_fixes.p'
output = BASEDIR+'dojo_simuser_output.p'

In [29]:
def VI(gt, seg):
    # total_vi = 0
    slice_vi = []    
    for i in range(len(gt)):
        current_vi = gp.Util.vi(gt[i].astype(np.int64), seg[i].astype(np.int64))
        # total_vi += current_vi
        slice_vi.append(current_vi)
    # total_vi /= 10
    return np.mean(slice_vi), np.median(slice_vi), slice_vi

def convert_vis(file, stopat=-1):
    vis = []
    with open(file, 'rb') as f:
        vis_ = pickle.load(f)    

    for i,vi in enumerate(vis_):
        
        if stopat != -1 and i>stopat:
            break        
        
        vis.append(vi[2])
    return vis

def convert_merge_fixes(file):
    fixes = []
    with open(file, 'rb') as f:
        fixes_ = pickle.load(f)
    for f in fixes_:
        if f=='Good' or f=='yes':
            fixes.append(('merge', '1'))
        elif f=='Bad' or f=='no':
            fixes.append(('merge', 'current'))
    return fixes

def convert_split_fixes(file, oracle=False, t=.95, stopat=-1):
    fixes = []
    with open(file, 'rb') as f:
        fixes_ = pickle.load(f)

    threshold_printed = False
    for i,f in enumerate(fixes_):
        
        if stopat != -1 and i>stopat:
            break
        
        if oracle==True:
#             print f[0]
            if f[0] == 1:
                fixes.append(('split', '1'))
            else:
                fixes.append(('split', 'current'))
        else:
            if f[1] < t and threshold_printed==False:
                print 'found threshold:', i-1
                threshold_printed = True
                
            fixes.append(('split', '1'))

    return fixes

def convert_user(output_folder, merge_vis, split_vis, merge_fixes, split_fixes, output, data='dojo', oracle=False, t=.95, stopat=-1, skipoutput=False):

    if data=='dojo':
        image, prob, gold, rhoana, bb = gp.Legacy.read_dojo_data()
    else:
        import glob, os
        import mahotas as mh
        golds = []
        rhoanas = []
        for z in range(250,300):
            path = '/home/d/data/cylinderNEW/'
            gold = sorted(glob.glob(os.path.join(path, 'gold', '*'+str(z)+'.png')))
            rhoana = sorted(glob.glob(os.path.join(path, 'rhoana', '*'+str(z)+'.png')))
            mask = sorted(glob.glob(os.path.join(path, 'mask', '*'+str(z)+'.png')))   
            mask = mh.imread(mask[0]).astype(np.bool)
            gold = mh.imread(gold[0])
            rhoana = mh.imread(rhoana[0])


            #convert ids from rgb to single channel
            rhoana_single = np.zeros((rhoana.shape[0], rhoana.shape[1]), dtype=np.uint64)
            rhoana_single[:, :] = rhoana[:,:,0]*256*256 + rhoana[:,:,1]*256 + rhoana[:,:,2]
            gold_single = np.zeros((gold.shape[0], gold.shape[1]), dtype=np.uint64)
            gold_single[:, :] = gold[:,:,0]*256*256 + gold[:,:,1]*256 + gold[:,:,2]

            # relabel the segmentations
            gold_single = gp.Util.relabel(gold_single)
            rhoana_single = gp.Util.relabel(rhoana_single)


            #mask the rhoana output
            rhoana_single[mask==0] = 0

            golds.append(gold_single)
            rhoanas.append(rhoana_single)
        gold = golds
        rhoana = rhoanas
    init_mean_vi = VI(gold, rhoana)[0]
    init_median_vi = VI(gold, rhoana)[1]
    init_vi_per_slice = VI(gold, rhoana)[2]    
    
    if merge_vis:
        c_merge_vis = convert_vis(merge_vis)
    else:
        c_merge_vis = []
    c_split_vis = convert_vis(split_vis, stopat=stopat)
    if merge_fixes:
        c_merge_fixes = convert_merge_fixes(merge_fixes)
    else:
        c_merge_fixes = []
    c_split_fixes = convert_split_fixes(split_fixes, oracle=oracle, t=t, stopat=stopat)
    
#     if not skipoutput:
#         with open(output, 'rb') as f:
#             output_seg = pickle.load(f)
    
    out_vis = [init_vi_per_slice]
    out_vis += c_merge_vis
    out_vis += c_split_vis
    
    out_fixes = []
    out_fixes += c_merge_fixes
    out_fixes += c_split_fixes
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    if not skipoutput:        
        os.rename(output, output_folder+'/ui_results.p')
#         with open(output_folder+'/ui_results.p', 'wb') as f:
#             pickle.dump(output_seg, f)
        
    with open(output_folder+'/times.p', 'wb') as f:
        # always 0
        pickle.dump([0], f)
        
    with open(output_folder+'/corrections.p', 'wb') as f:
        pickle.dump(out_fixes, f)
        
    with open(output_folder+'/correction_vis.p', 'wb') as f:
        pickle.dump(out_vis, f)
    

In [104]:
BASEDIR = '/home/d/netstatsPAPER/IPMLB/'
OUTDIR = '/home/d/GPSTUDY/'
merge_vis = BASEDIR+'dojo_merge_simuser_vis.p'
split_vis = BASEDIR+'dojo_split_simuser_vis.p'
merge_fixes = BASEDIR+'dojo_merge_simuser_fixes.p'
split_fixes = BASEDIR+'dojo_split_simuser_fixes.p'
output = BASEDIR+'dojo_simuser_output.p'

convert_user(OUTDIR+'simuserGP_NEW', merge_vis, split_vis, merge_fixes, split_fixes, output, oracle=True)

In [105]:
BASEDIR = '/home/d/netstatsPAPER/IPMLB/'
OUTDIR = '/home/d/GPSTUDY/'
merge_vis = BASEDIR+'dojo_merge_till_end_vis.p'
split_vis = BASEDIR+'dojo_split_till_end_vis.p'
merge_fixes = BASEDIR+'dojo_merge_till_end_fixes.p'
split_fixes = BASEDIR+'dojo_split_till_end_fixes.p'
output = BASEDIR+'dojo_till_end_output.p'

convert_user(OUTDIR+'auto95GP_NEW', merge_vis, split_vis, merge_fixes, split_fixes, output)

found threshold: 153


In [87]:
BASEDIR = '/home/d/netstatsPAPERFP/IPMLB/'
OUTDIR = '/home/d/GPSTUDY/'
merge_vis = None
split_vis = BASEDIR+'dojo_split_simuser_vis.p'
merge_fixes = None
split_fixes = BASEDIR+'dojo_split_simuser_fixes.p'
output = BASEDIR+'dojo_simuser_output.p'

convert_user(OUTDIR+'simuserFP_NEW', merge_vis, split_vis, merge_fixes, split_fixes, output, oracle=True)

In [91]:
BASEDIR = '/home/d/netstatsPAPERFP/IPMLB/'
OUTDIR = '/home/d/GPSTUDY/'
merge_vis = None
split_vis = BASEDIR+'dojo_split_auto95_vis.p'
merge_fixes = None
split_fixes = BASEDIR+'dojo_split_auto95_fixes.p'
output = BASEDIR+'dojo_auto95_output.p'

convert_user(OUTDIR+'auto95FP_NEW', merge_vis, split_vis, merge_fixes, split_fixes, output)

found threshold: -1


In [103]:
BASEDIR = '/home/d/netstatsPAPER/IPMLB/'
OUTDIR = '/home/d/GPSTUDY/'
merge_vis = BASEDIR+'dojo_merge_till_end_vis.p'
split_vis = BASEDIR+'dojo_split_till_end_vis.p'
merge_fixes = BASEDIR+'dojo_merge_till_end_fixes.p'
split_fixes = BASEDIR+'dojo_split_till_end_fixes.p'
output = BASEDIR+'dojo_till_end_output.p'

convert_user(OUTDIR+'auto95GP_threshold_NEW', merge_vis, split_vis, merge_fixes, split_fixes, output, stopat=100)

found threshold: 75


In [100]:
BASEDIR = '/home/d/netstatsPAPERFP/IPMLB/'
OUTDIR = '/home/d/GPSTUDY/'
merge_vis = None
split_vis = BASEDIR+'dojo_split_auto95_vis.p'
merge_fixes = None
split_fixes = BASEDIR+'dojo_split_auto95_fixes.p'
output = BASEDIR+'dojo_auto95_output.p'

convert_user(OUTDIR+'auto95FP_threshold_NEW', merge_vis, split_vis, merge_fixes, split_fixes, output, stopat=100)

found threshold: -1


In [30]:
BASEDIR = '/home/d/CYLINDERSTUDY/cylinderFP/'
OUTDIR = '/home/d/CYLINDERSTUDY/'
merge_vis = None
split_vis = BASEDIR+'cylinder_split_simuser_vis.p'
merge_fixes = None
split_fixes = BASEDIR+'cylinder_split_simuser_fixes.p'
output = BASEDIR+'cylinder_simuser_output.p'

convert_user(OUTDIR+'simuserFP_NEW', merge_vis, split_vis, merge_fixes, split_fixes, output, 
             oracle=True, data='cylinder')
print 'Done'

Done


In [31]:
BASEDIR = '/home/d/CYLINDERSTUDY/cylinderFP/'
OUTDIR = '/home/d/CYLINDERSTUDY/'
merge_vis = None
split_vis = BASEDIR+'cylinder_split_auto95_vis.p'
merge_fixes = None
split_fixes = BASEDIR+'cylinder_split_auto95_fixes.p'
output = BASEDIR+'cylinder_auto95_output.p'

convert_user(OUTDIR+'auto95FP_NEW', merge_vis, split_vis, merge_fixes, split_fixes, output, 
             oracle=False, data='cylinder')
print 'Done'

found threshold: 1051
Done


In [3]:
BASEDIR = '/home/d/CYLINDERSTUDY/cylinderGP/'
OUTDIR = '/home/d/CYLINDERSTUDY/'
merge_vis = BASEDIR+'cylinder_merge_simuser_vis.p'
split_vis = BASEDIR+'cylinder_split_simuser_vis.p'
merge_fixes = BASEDIR+'cylinder_merge_simuser_fixes.p'
split_fixes = BASEDIR+'cylinder_split_simuser_fixes.p'
output = BASEDIR+'cylinder_simuser_output.p'

convert_user(OUTDIR+'simuserGP_NEW', merge_vis, split_vis, merge_fixes, split_fixes, output, oracle=True)
print 'Done'

Done


In [4]:
BASEDIR = '/home/d/CYLINDERSTUDY/cylinderGP/'
OUTDIR = '/home/d/CYLINDERSTUDY/'
merge_vis = BASEDIR+'cylinder_merge_auto00_vis.p'
split_vis = BASEDIR+'cylinder_split_auto00_vis.p'
merge_fixes = BASEDIR+'cylinder_merge_auto00_fixes.p'
split_fixes = BASEDIR+'cylinder_split_auto00_fixes.p'
output = BASEDIR+'cylinder_auto00_output.p'

convert_user(OUTDIR+'auto00GP_NEW', merge_vis, split_vis, merge_fixes, split_fixes, output)
print 'Done'

found threshold: 2160
Done
